## Prep

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

import statsmodels.api as sm
from scipy import stats
import statsmodels.tools as tools

import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 100)

In [ ]:
mesa = pd.read_csv('../mesa/data_processed/Y_BaselineX_processed_full.csv')

mesa = mesa.rename(columns = {'cvda': 'Y_tot', 
                             'F1_PC2': 'nSES',
                            'S1FAV': 'nFavFood',
                            'S1PAI': 'nPhysFac', 
                            'G_bla_rk': 'nRS', 
                            'chdiet': 'nutrition', 
                            'chphysact': 'PhysAct',
                            'income': 'FamIncome'})

diet_phys_map = {0:1, 1:2, 2:3}
mesa['nutrition'] = mesa['nutrition'].replace(diet_phys_map)
mesa['PhysAct'] = mesa['PhysAct'].replace(diet_phys_map)

mesa_bla = mesa[mesa['race'] == 3]

In [ ]:
jhs = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs = jhs.drop("nSES", axis=1)

jhs = jhs.rename(columns = {'event': 'Y_tot', 
                             'nbSESpc2score': 'nSES',
                            'S1FAV': 'nFavFood',
                            'S1PAI': 'nPhysFac', 
                            'G_bla_rk': 'nRS', 
                            'nutrition3cat': 'nutrition', 
                            'PA3cat': 'PhysAct',
                            'fmlyinc': 'FamIncome'})
jhs['nutrition'] = jhs['nutrition'].replace(diet_phys_map)
jhs['PhysAct'] = jhs['PhysAct'].replace(diet_phys_map)

## Mesa

In [ ]:
y = mesa['Y_tot']

X = mesa[['nSES','nFavFood','nPhysFac', 'nRS',
          'FamIncome','nutrition', 'PhysAct',
          'age','gender','race','cig','cural','diabet','hdl','chol','sbp']]

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()
X4 = X.copy()
X5 = X.copy()
X6 = X.copy()

X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
X1 = pd.get_dummies(X1, columns=['gender','race','cig','cural','diabet'], 
                    drop_first=True)

X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']

In [ ]:
logistic_model_with_moderation = sm.Logit(y, sm.add_constant(X1)).fit()
print(logistic_model_with_moderation.summary())

## MESA BLACK

In [ ]:
y = mesa_bla['Y_tot']

X = mesa_bla[['nSES','nFavFood','nPhysFac', 'nRS',
          'FamIncome','nutrition', 'PhysAct',
          'age','gender','race','cig','cural','diabet','hdl','chol','sbp']]

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()
X4 = X.copy()
X5 = X.copy()
X6 = X.copy()

X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
# X1 = pd.get_dummies(X1, columns=['gender','race','cig','cural','diabet'], 
#                     drop_first=True)
X1 = pd.get_dummies(X1, columns=['nSES','nFavFood','nPhysFac', 'nRS',
                                 'FamIncome','nutrition', 'PhysAct',
                                 'gender','race','cig','cural','diabet','nSES_FamIncome'], 
                    drop_first=True)

X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']

In [ ]:
logistic_model_with_moderation = sm.Logit(y, sm.add_constant(X1)).fit()
print(logistic_model_with_moderation.summary())

## JHS

In [ ]:
y = jhs['Y_tot']

X = jhs[['nSES','nFavFood','nPhysFac', 'nRS',
          'FamIncome','nutrition', 'PhysAct',
          'age','gender','currentSmoker','alc','Diabetes','hdl','totchol','sbp']]

X1 = X.copy()
X2 = X.copy()
X3 = X.copy()
X4 = X.copy()
X5 = X.copy()
X6 = X.copy()

X1['nSES_FamIncome'] = X['nSES'] * X['FamIncome']
# X1 = pd.get_dummies(X1, columns=['gender','currentSmoker','alc','Diabetes'], 
#                     drop_first=True)
X1 = pd.get_dummies(X1, columns=['nSES','nFavFood','nPhysFac', 'nRS',
                                 'FamIncome','nutrition', 'PhysAct',
                                 'gender','currentSmoker','alc','Diabetes','nSES_FamIncome'], 
                    drop_first=True)


X2['nFavFood_FamIncome'] = X['nFavFood'] * X['FamIncome']
X3['nPhysFac_FamIncome'] = X['nPhysFac'] * X['FamIncome']
X4['nRS_FamIncome'] = X['nRS'] * X['FamIncome']
X5['nFavFood_nutrition'] = X['nFavFood'] * X['nutrition']
X6['nPhysFac_PhysAct'] = X['nPhysFac'] * X['PhysAct']

In [ ]:
logistic_model_with_moderation = sm.Logit(y, sm.add_constant(X1)).fit()
print(logistic_model_with_moderation.summary())